In [ ]:
%%bash
wget https://github.com/diffblue/cbmc/releases/download/cbmc-5.95.1/ubuntu-20.04-cbmc-5.95.1-Linux.deb
dpkg -i ubuntu-20.04-cbmc-5.95.1-Linux.deb

CBMC : Shamefully underused

I'm very impressed by the SV-Comp style solvers, in particular [CBMC](https://github.com/diffblue/cbmc) [Documentation](https://diffblue.github.io/cbmc//index.html).

I thought CBMC might be defunct, since it's main webpage does not show recent development, but actually the github repo is very active. It appears Amazon is investing n it's usage with a number of significant C projects having CBMC based specs ad verification.

It's pretty easy to install and use.

Here's a basic example:


In [2]:
%%file /tmp/test.c
#include <assert.h>
#include <stdint.h>

int64_t myabs(int64_t x){
    return x < 0 ? -x : x;
}

int64_t nondet_int64();

int main(){
    int64_t x = nondet_int();
    int64_t y = myabs(x);
    assert(y >= 0);
}

Writing /tmp/test.c


In [3]:
!cbmc /tmp/test.c

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/test.c
Converting
Type-checking test
file /tmp/test.c line 11 function main: function 'nondet_int' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Runtime Symex: 0.00335992s
size of program expression: 33 steps
simple slicing removed 5 assignments
Generated 1 VCC(s), 1 remaining after simplification
Runtime Postprocess Equation: 1.4824e-05s
Passing problem to propositional reduction
converting SSA
Runtime Convert SSA: 0.00153247s
Running propositional reduction
Post-processing
Runtime Post-process: 3.28e-06s
Solving with MiniSAT 2.2.1 with simplifier
637 variables, 1074 clauses
SAT checker: instance is UNSATISFIABLE
Runtime Solver: 0.000218563s
Runtime decision procedure: 0.00218171s

** Results:
/tmp/test.c function main
[main.as

## Comparative Checking

Comparative checking of programs can be useful for reducing both the specification and verification burden.
In addition, there are some relational properties ("hyperproperties") that require talking about two different runs of a program. The canonical example is information security. If you run two programs with identical low security parts of the states, they must end with identical low security parts of the state. This is true if there is only influence or information transferral going from low to high.

- Translation validation
- Refinement
- Security properties
- Bug fixing

The simplest approach to doing comparative verification so is to use the more commonly available single program verifier on the "product program".

```bash
echo '
#include <assert.h>
int safeprog(int low, int high){
    int foo = low ^ high;
    foo = ((foo << 1) ^ high) >> 1;
    return foo;
}

int main(){
    int high = nondet_int();
    int high1 = nondet_int();
    int low = nondet_int();
    //int high, high1, low;

    __ESBMC_assert(safeprog(low,high) == safeprog(low,high1), "information security property");
    return 0;
}
' > /tmp/test.c
esbmc /tmp/test.c
```

There are many bits of software out there that have security vulnerabilities.

Obviously, the easiest and most likely solution is to change the code in source, recompile, and push out the new version.
This is not always possible or desirable however.

- Even recompiling the original program may introduce bugs due to compiler difference versions
- You may not have the source
-

It is in general an interesting problem to consdier the 50 year software stack.

Worse is better: A use case for dumb compilers


# Bits and Bobbles

[esbmc](http://esbmc.org/) is a C bounded model checker.

Comparing a C function to it's grammar?

```bash
echo "
#include<stdbool.h>

bool check_balance(char *input){
    int count = 0;
    while(*input != '\0'){
        if(*input == '(') count++;
        if(*input == ')') count--;
        input++;
    }
    return count == 0;
" > /tmp/parens.c
cbmc /tmp/parens.c 
```

In [11]:
%%script sqlite3 --echo
CREATE TABLE test (x INTEGER, y INTEGER);
INSERT INTO test VALUES (1, 2);
INSERT INTO test SELECT x+1, y+1 FROM test;
SELECT * FROM test;
.quit

1|2
2|3


In [12]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [13]:
%quickref


IPython -- An enhanced Interactive Python - Quick Reference Card

obj?, obj??      : Get help, or more help for object (also works as
                   ?obj, ??obj).
?foo.*abc*       : List names in 'foo' containing 'abc' in them.
%magic           : Information about IPython's 'magic' % functions.

Magic functions are prefixed by % or %%, and typically take their arguments
without parentheses, quotes or even commas for convenience.  Line magics take a
single % and cell magics are prefixed with two %%.

Example magic function calls:

%alias d ls -F   : 'd' is now an alias for 'ls -F'
alias d ls -F    : Works if 'alias' not a python name
alist = %alias   : Get list of aliases to 'alist'
cd /usr/share    : Obvious. cd -<tab> to choose from visited dirs.
%cd??            : See help AND source for magic %cd
%timeit x=10     : time the 'x=10' statement with high precision.
%%timeit x=2**100
x**100           : time 'x**100' with a setup of 'x=2**100'; setup code is not
                   co

In [16]:
%script z3
(echo "starting z3")
(declare-const x Int)
(declare-const y Int)
(assert (= y (+ x 1)))
(check-sat)
(get-model)

SyntaxError: invalid syntax (3432626410.py, line 2)